# State by State Daily Neg Ad Mix by Party - 7DMA

In [1]:
import pprint
import pickle
import pandas as pd
from datetime import datetime
import numpy as np
import string
from collections import Counter
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

pd.set_option("display.max_columns", 2000)
pd.set_option("display.max_rows", 2000)
pd.set_option("display.precision", 2)
# pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Using 'fivethirtyeight' style sheet

plt.style.use('fivethirtyeight')

In [3]:
# reading in cleaned up dataset

cand_date_df = pd.read_pickle('data/pres_sorted_with_sponsors_and_party.pickle')
cand_date_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count,hour_aired,sponsor_class,party
start_time,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-01 01:19:59,261456,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",On the Record With Greta Van Susteren,news,2016-08-01 01:19:59,2016-08-01 01:21:04,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,2016.0,Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.0,Monday,San Francisco-Oakland-San Jose,National,1.0,1,R_PAC,R
2016-08-01 06:20:04,261450,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",Fox News Sunday With Chris Wallace,news,2016-08-01 06:20:04,2016-08-01 06:21:09,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,2016.0,Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.0,Monday,San Francisco-Oakland-San Jose,National,1.0,6,R_PAC,R
2016-08-01 08:39:49,48222,4176,WCAU,"Philadelphia, PA",NBC10 News Today at 430am,news,2016-08-01 08:39:49,2016-08-01 08:40:21,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,2016.0,"Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.0,Monday,Philadelphia,PA,4.0,8,Clinton,D
2016-08-01 08:48:14,48395,4176,KYW,"Philadelphia, PA",Eyewitness News at 430,news,2016-08-01 08:48:14,2016-08-01 08:48:46,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,2016.0,"Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.0,Monday,Philadelphia,PA,4.0,8,Clinton,D
2016-08-01 09:09:45,378441,7459,MSNBCW,"San Francisco-Oakland-San Jose, CA",First Look,news,2016-08-01 09:09:45,2016-08-01 09:10:15,PolAd_HillaryClinton_4gvic,https://archive.org/embed/PolAd_HillaryClinton...,Make America Number One,Super PAC,PRES,2016.0,Candidate Biography,Hillary Clinton,campaign,con,2016-10-21 10:40:01,30.0,Monday,San Francisco-Oakland-San Jose,National,1.0,9,R_PAC,R


In [4]:
cand_date_df['state'].value_counts()

FL          12284
PA          11915
NV          10928
OH          10322
National     7217
NC           6722
IA           5284
NH           4963
CO           4689
WI           4065
AZ           3219
VA           1695
NY             46
Name: state, dtype: int64

In [5]:
# getting rid of NY

cand_date_df = cand_date_df[cand_date_df['state'] != 'NY']

In [6]:
state_list = list(cand_date_df['state'].unique())
state_list

['National', 'PA', 'NV', 'NC', 'FL', 'NH', 'CO', 'OH', 'AZ', 'WI', 'VA', 'IA']

In [7]:
# creating a dictionary of DataSeries
# key = Series Name, value = Count of Ads

state_DataFrames = {}
party_list = ['D', 'R']

for state in state_list:
    for party_initial in party_list:
        state_DataFrames[str(state+'_'+party_initial+'_con')] = cand_date_df.copy()['party'][(cand_date_df['party'] == party_initial) & 
                                                                                             (cand_date_df['message'] == 'con') &
                                                                                             (cand_date_df['state'] == state)].resample('D').count()
        
        state_DataFrames[str(state+'_'+party_initial+'_tot')] = cand_date_df.copy()['party'][(cand_date_df['party'] == party_initial) & 
                                                                                             (cand_date_df['state'] == state)].resample('D').count()
        
        state_DataFrames[str(state+'_'+party_initial+'_neg_ratio')] = ((state_DataFrames[str(state+'_'+party_initial+'_con')] /
                                                                       state_DataFrames[str(state+'_'+party_initial+'_tot')]) * 100)

In [8]:
state_DataFrames.keys()

dict_keys(['FL_R_con', 'NV_R_tot', 'NH_R_neg_ratio', 'WI_D_con', 'VA_D_neg_ratio', 'WI_D_tot', 'OH_R_con', 'IA_D_tot', 'FL_R_tot', 'VA_R_neg_ratio', 'NC_D_con', 'NC_D_tot', 'NC_R_con', 'WI_R_tot', 'NV_D_neg_ratio', 'PA_R_con', 'PA_R_neg_ratio', 'CO_R_tot', 'AZ_D_neg_ratio', 'IA_R_tot', 'WI_D_neg_ratio', 'NV_D_con', 'WI_R_neg_ratio', 'CO_D_neg_ratio', 'IA_D_neg_ratio', 'PA_D_tot', 'NV_R_neg_ratio', 'FL_D_tot', 'NH_D_con', 'NC_D_neg_ratio', 'NH_D_neg_ratio', 'VA_D_tot', 'WI_R_con', 'OH_D_neg_ratio', 'National_R_tot', 'VA_R_tot', 'NH_D_tot', 'OH_R_tot', 'CO_R_neg_ratio', 'NC_R_tot', 'NH_R_tot', 'NC_R_neg_ratio', 'National_D_con', 'National_R_con', 'National_D_tot', 'AZ_R_con', 'CO_D_con', 'IA_R_con', 'National_D_neg_ratio', 'FL_D_neg_ratio', 'IA_R_neg_ratio', 'PA_D_con', 'National_R_neg_ratio', 'FL_R_neg_ratio', 'NV_R_con', 'OH_D_con', 'CO_D_tot', 'NV_D_tot', 'PA_R_tot', 'OH_D_tot', 'IA_D_con', 'AZ_D_tot', 'AZ_R_tot', 'PA_D_neg_ratio', 'VA_R_con', 'AZ_R_neg_ratio', 'CO_R_con', 'FL_D_con',

In [9]:
# Note the differing start dates between the 'OH_R_con' Series and 'OH_D_con' Series
# That is why have to put all in the same dataframe to harmonize dates for taking the rolling mean

state_DataFrames['OH_R_con'].head(30)

start_time
2016-08-08    22
2016-08-09    28
2016-08-10    26
2016-08-11    30
2016-08-12    30
2016-08-13     7
2016-08-14     8
2016-08-15    30
2016-08-16    54
2016-08-17    59
2016-08-18    58
2016-08-19    56
2016-08-20    29
2016-08-21    27
2016-08-22    60
2016-08-23    54
2016-08-24    60
2016-08-25    19
2016-08-26    45
2016-08-27    35
2016-08-28    24
2016-08-29    48
2016-08-30     8
2016-08-31     0
2016-09-01     0
2016-09-02     0
2016-09-03     1
2016-09-04     0
2016-09-05     0
2016-09-06     0
Freq: D, Name: party, dtype: int64

In [10]:
state_DataFrames['OH_D_con'].head(30)

start_time
2016-08-07     5
2016-08-08    51
2016-08-09    65
2016-08-10    54
2016-08-11    41
2016-08-12    46
2016-08-13    13
2016-08-14    35
2016-08-15    65
2016-08-16    31
2016-08-17    21
2016-08-18    17
2016-08-19    17
2016-08-20     5
2016-08-21    12
2016-08-22    24
2016-08-23    44
2016-08-24    38
2016-08-25    29
2016-08-26    29
2016-08-27    11
2016-08-28    12
2016-08-29    57
2016-08-30    63
2016-08-31    64
2016-09-01    50
2016-09-02    46
2016-09-03    10
2016-09-04    12
2016-09-05    26
Freq: D, Name: party, dtype: int64

In [11]:
state_keys_list = sorted(list(state_DataFrames.keys()))
state_keys_list

['AZ_D_con',
 'AZ_D_neg_ratio',
 'AZ_D_tot',
 'AZ_R_con',
 'AZ_R_neg_ratio',
 'AZ_R_tot',
 'CO_D_con',
 'CO_D_neg_ratio',
 'CO_D_tot',
 'CO_R_con',
 'CO_R_neg_ratio',
 'CO_R_tot',
 'FL_D_con',
 'FL_D_neg_ratio',
 'FL_D_tot',
 'FL_R_con',
 'FL_R_neg_ratio',
 'FL_R_tot',
 'IA_D_con',
 'IA_D_neg_ratio',
 'IA_D_tot',
 'IA_R_con',
 'IA_R_neg_ratio',
 'IA_R_tot',
 'NC_D_con',
 'NC_D_neg_ratio',
 'NC_D_tot',
 'NC_R_con',
 'NC_R_neg_ratio',
 'NC_R_tot',
 'NH_D_con',
 'NH_D_neg_ratio',
 'NH_D_tot',
 'NH_R_con',
 'NH_R_neg_ratio',
 'NH_R_tot',
 'NV_D_con',
 'NV_D_neg_ratio',
 'NV_D_tot',
 'NV_R_con',
 'NV_R_neg_ratio',
 'NV_R_tot',
 'National_D_con',
 'National_D_neg_ratio',
 'National_D_tot',
 'National_R_con',
 'National_R_neg_ratio',
 'National_R_tot',
 'OH_D_con',
 'OH_D_neg_ratio',
 'OH_D_tot',
 'OH_R_con',
 'OH_R_neg_ratio',
 'OH_R_tot',
 'PA_D_con',
 'PA_D_neg_ratio',
 'PA_D_tot',
 'PA_R_con',
 'PA_R_neg_ratio',
 'PA_R_tot',
 'VA_D_con',
 'VA_D_neg_ratio',
 'VA_D_tot',
 'VA_R_con',
 'VA_R

In [12]:
# Putting new DataSeries into a list -- need to do so in order to use pd.concat to join them together in a DataFrame

state_DataFrames_list = []

for item in state_keys_list:
    state_DataFrames_list.append(state_DataFrames[item])

In [13]:
# Making sure new list has all the dataframes

len(state_keys_list), len(state_DataFrames_list)

(72, 72)

In [14]:
# Using pd.concat to join individual series in a list in one DataFrame - harmonize start dates

state_party_df = pd.concat(state_DataFrames_list, axis=1, join='outer')
state_party_df.columns = state_keys_list
state_party_df.head(20)

,AZ_D_con,AZ_D_neg_ratio,AZ_D_tot,AZ_R_con,AZ_R_neg_ratio,AZ_R_tot,CO_D_con,CO_D_neg_ratio,CO_D_tot,CO_R_con,CO_R_neg_ratio,CO_R_tot,FL_D_con,FL_D_neg_ratio,FL_D_tot,FL_R_con,FL_R_neg_ratio,FL_R_tot,IA_D_con,IA_D_neg_ratio,IA_D_tot,IA_R_con,IA_R_neg_ratio,IA_R_tot,NC_D_con,NC_D_neg_ratio,NC_D_tot,NC_R_con,NC_R_neg_ratio,NC_R_tot,NH_D_con,NH_D_neg_ratio,NH_D_tot,NH_R_con,NH_R_neg_ratio,NH_R_tot,NV_D_con,NV_D_neg_ratio,NV_D_tot,NV_R_con,NV_R_neg_ratio,NV_R_tot,National_D_con,National_D_neg_ratio,National_D_tot,National_R_con,National_R_neg_ratio,National_R_tot,OH_D_con,OH_D_neg_ratio,OH_D_tot,OH_R_con,OH_R_neg_ratio,OH_R_tot,PA_D_con,PA_D_neg_ratio,PA_D_tot,PA_R_con,PA_R_neg_ratio,PA_R_tot,VA_D_con,VA_D_neg_ratio,VA_D_tot,VA_R_con,VA_R_neg_ratio,VA_R_tot,WI_D_con,WI_D_neg_ratio,WI_D_tot,WI_R_con,WI_R_neg_ratio,WI_R_tot
start_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,25.00,4,22,95.65,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,5,83.33,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,58.33,12,18,90.00,20,NaN,NaN,NaN,NaN,NaN,NaN,35.0,50.72,69,4,80.00,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,57.14,21,16,94.12,17,NaN,NaN,NaN,NaN,NaN,NaN,48.0,59.26,81,4,100.00,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,100.00,1.0,NaN,NaN,NaN,3,37.50,8,9,90.00,10,NaN,NaN,NaN,NaN,NaN,NaN,39.0,72.22,54,3,100.00,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,100.00,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,100.00,12.0,NaN,NaN,NaN,1.0,100.00,1.0,NaN,NaN,NaN,8.0,80.00,10.0,NaN,NaN,NaN,7,50.00,14,5,83.33,6,NaN,NaN,NaN,NaN,NaN,NaN,32.0,91.43,35,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,15.0,71.43,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,83.33,12.0,NaN,NaN,NaN,4.0,66.67,6.0,NaN,NaN,NaN,18.0,64.29,28.0,NaN,NaN,NaN,9,75.00,12,13,92.86,14,NaN,NaN,NaN,NaN,NaN,NaN,18.0,94.74,19,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,100.00,2.0,NaN,NaN,NaN,2.0,100.00,2.0,NaN,NaN,NaN,5.0,100.00,5.0,1.0,100.0,1.0,5,83.33,6,3,100.00,3,5.0,100.00,5.0,NaN,NaN,NaN,19.0,100.00,19,2,100.00,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-08,4.0,80.00,5.0,NaN,NaN,NaN,1.0,100.00,1.0,NaN,NaN,2.0,26.0,68.42,38.0,15.0,75.00,20.0,NaN,NaN,NaN,NaN,NaN,NaN,26.0,83.87,31.0,7.0,70.00,10.0,4.0,66.67,6.0,NaN,NaN,NaN,28.0,77.78,36.0,0.0,NaN,0.0,15,78.95,19,18,45.00,40,51.0,85.00,60.0,22.0,84.62,26.0,33.0,70.21,47,0,0.00,1,NaN,NaN,NaN,NaN,NaN,NaN,2.0,66.67,3.0,NaN,NaN,NaN
2016-08-09,1.0,33.33,3.0,NaN,NaN,1.0,0.0,0.00,2.0,NaN,NaN,2.0,35.0,64.81,54.0,15.0,100.00,15.0,NaN,NaN,NaN,NaN,NaN,NaN,18.0,75.00,24.0,11.0,64.71,17.0,10.0,83.33,12.0,NaN,NaN,1.0,50.0,72.46,69.0,0.0,0.0,2.0,30,85.71,35,18,52.94,34,65.0,80.25,81.0,28.0,84.85,33.0,48.0,57.14,84,0,0.00,2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,50.00,2.0,NaN,NaN,NaN


In [15]:
# Now that all of the states have the same start dates -- fillna with zero then take the rolling averages

state_party_df.fillna(0, inplace=True)
state_party_df.head(20)

,AZ_D_con,AZ_D_neg_ratio,AZ_D_tot,AZ_R_con,AZ_R_neg_ratio,AZ_R_tot,CO_D_con,CO_D_neg_ratio,CO_D_tot,CO_R_con,CO_R_neg_ratio,CO_R_tot,FL_D_con,FL_D_neg_ratio,FL_D_tot,FL_R_con,FL_R_neg_ratio,FL_R_tot,IA_D_con,IA_D_neg_ratio,IA_D_tot,IA_R_con,IA_R_neg_ratio,IA_R_tot,NC_D_con,NC_D_neg_ratio,NC_D_tot,NC_R_con,NC_R_neg_ratio,NC_R_tot,NH_D_con,NH_D_neg_ratio,NH_D_tot,NH_R_con,NH_R_neg_ratio,NH_R_tot,NV_D_con,NV_D_neg_ratio,NV_D_tot,NV_R_con,NV_R_neg_ratio,NV_R_tot,National_D_con,National_D_neg_ratio,National_D_tot,National_R_con,National_R_neg_ratio,National_R_tot,OH_D_con,OH_D_neg_ratio,OH_D_tot,OH_R_con,OH_R_neg_ratio,OH_R_tot,PA_D_con,PA_D_neg_ratio,PA_D_tot,PA_R_con,PA_R_neg_ratio,PA_R_tot,VA_D_con,VA_D_neg_ratio,VA_D_tot,VA_R_con,VA_R_neg_ratio,VA_R_tot,WI_D_con,WI_D_neg_ratio,WI_D_tot,WI_R_con,WI_R_neg_ratio,WI_R_tot
start_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-01,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1,25.00,4,22,95.65,23,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,18,5,83.33,6,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2016-08-02,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,7,58.33,12,18,90.00,20,0.0,0.00,0.0,0.0,0.00,0.0,35.0,50.72,69,4,80.00,5,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2016-08-03,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,12,57.14,21,16,94.12,17,0.0,0.00,0.0,0.0,0.00,0.0,48.0,59.26,81,4,100.00,4,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2016-08-04,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1.0,100.00,1.0,0.0,0.0,0.0,3,37.50,8,9,90.00,10,0.0,0.00,0.0,0.0,0.00,0.0,39.0,72.22,54,3,100.00,3,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2016-08-05,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5.0,100.00,5.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,100.00,12.0,0.0,0.00,0.0,1.0,100.00,1.0,0.0,0.0,0.0,8.0,80.00,10.0,0.0,0.0,0.0,7,50.00,14,5,83.33,6,0.0,0.00,0.0,0.0,0.00,0.0,32.0,91.43,35,0,0.00,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2016-08-06,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,2.0,0.0,0.0,0.0,15.0,71.43,21.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,83.33,12.0,0.0,0.00,0.0,4.0,66.67,6.0,0.0,0.0,0.0,18.0,64.29,28.0,0.0,0.0,0.0,9,75.00,12,13,92.86,14,0.0,0.00,0.0,0.0,0.00,0.0,18.0,94.74,19,0,0.00,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2016-08-07,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,100.00,2.0,0.0,0.00,0.0,2.0,100.00,2.0,0.0,0.0,0.0,5.0,100.00,5.0,1.0,100.0,1.0,5,83.33,6,3,100.00,3,5.0,100.00,5.0,0.0,0.00,0.0,19.0,100.00,19,2,100.00,2,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2016-08-08,4.0,80.00,5.0,0.0,0.0,0.0,1.0,100.00,1.0,0.0,0.0,2.0,26.0,68.42,38.0,15.0,75.00,20.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,83.87,31.0,7.0,70.00,10.0,4.0,66.67,6.0,0.0,0.0,0.0,28.0,77.78,36.0,0.0,0.0,0.0,15,78.95,19,18,45.00,40,51.0,85.00,60.0,22.0,84.62,26.0,33.0,70.21,47,0,0.00,1,0.0,0.00,0.0,0.0,0.0,0.0,2.0,66.67,3.0,0.0,0.0,0.0
2016-08-09,1.0,33.33,3.0,0.0,0.0,1.0,0.0,0.00,2.0,0.0,0.0,2.0,35.0,64.81,54.0,15.0,100.00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,75.00,24.0,11.0,64.71,17.0,10.0,83.33,12.0,0.0,0.0,1.0,50.0,72.46,69.0,0.0,0.0,2.0,30,85.71,35,18,52.94,34,65.0,80.25,81.0,28.0,84.85,33.0,48.0,57.14,84,0,0.00,2,0.0,0.00,0.0,0.0,0.0,0.0,1.0,50.00,2.0,0.0,0.0,0.0


In [16]:
state_party_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2016-08-01 to 2016-11-08
Freq: D
Data columns (total 72 columns):
AZ_D_con                100 non-null float64
AZ_D_neg_ratio          100 non-null float64
AZ_D_tot                100 non-null float64
AZ_R_con                100 non-null float64
AZ_R_neg_ratio          100 non-null float64
AZ_R_tot                100 non-null float64
CO_D_con                100 non-null float64
CO_D_neg_ratio          100 non-null float64
CO_D_tot                100 non-null float64
CO_R_con                100 non-null float64
CO_R_neg_ratio          100 non-null float64
CO_R_tot                100 non-null float64
FL_D_con                100 non-null float64
FL_D_neg_ratio          100 non-null float64
FL_D_tot                100 non-null float64
FL_R_con                100 non-null float64
FL_R_neg_ratio          100 non-null float64
FL_R_tot                100 non-null float64
IA_D_con                100 non-null float64
IA_D_neg_rati

In [17]:
state_party_df.shape

(100, 72)

In [18]:
# make sure these numbers match DataFrame below for '2016-08-07'

state_party_df['National_D_neg_ratio']['2016-08-01':'2016-08-07'].sum() / 7

55.18707482993198

In [19]:
state_party_df['National_R_neg_ratio']['2016-08-01':'2016-08-07'].sum() / 7

92.280042451763322

In [20]:
# Can apply the .rolling() to a whole dataframe -- which is convenient -- versus doing one series at a time
# Only time you get a NaN when there is not enough observations at the beginning of the dataset
# hence there will be 6 NaNs for each column -- see .info() below
# when you do the .rolling() on this DataFrame, the con and tot columns become non-sensical -- which is OK b/c
# not using them anyway in the charts

state_party_df = state_party_df.rolling(7).mean()

In [21]:
state_party_df['2016-08-07':'2016-09-01']

,AZ_D_con,AZ_D_neg_ratio,AZ_D_tot,AZ_R_con,AZ_R_neg_ratio,AZ_R_tot,CO_D_con,CO_D_neg_ratio,CO_D_tot,CO_R_con,CO_R_neg_ratio,CO_R_tot,FL_D_con,FL_D_neg_ratio,FL_D_tot,FL_R_con,FL_R_neg_ratio,FL_R_tot,IA_D_con,IA_D_neg_ratio,IA_D_tot,IA_R_con,IA_R_neg_ratio,IA_R_tot,NC_D_con,NC_D_neg_ratio,NC_D_tot,NC_R_con,NC_R_neg_ratio,NC_R_tot,NH_D_con,NH_D_neg_ratio,NH_D_tot,NH_R_con,NH_R_neg_ratio,NH_R_tot,NV_D_con,NV_D_neg_ratio,NV_D_tot,NV_R_con,NV_R_neg_ratio,NV_R_tot,National_D_con,National_D_neg_ratio,National_D_tot,National_R_con,National_R_neg_ratio,National_R_tot,OH_D_con,OH_D_neg_ratio,OH_D_tot,OH_R_con,OH_R_neg_ratio,OH_R_tot,PA_D_con,PA_D_neg_ratio,PA_D_tot,PA_R_con,PA_R_neg_ratio,PA_R_tot,VA_D_con,VA_D_neg_ratio,VA_D_tot,VA_R_con,VA_R_neg_ratio,VA_R_tot,WI_D_con,WI_D_neg_ratio,WI_D_tot,WI_R_con,WI_R_neg_ratio,WI_R_tot
start_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.29,0.00,0.00,0.00,2.86,24.49,3.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.43,40.48,3.71,0.00,0.00,0.00,1.00,38.10,1.29,0.0,0.0,0.00,4.57,49.18,6.29,0.14,14.29,0.14,6.29,55.19,11.00,12.29,92.28,13.29,0.71,14.29,0.71,0.00,0.00,0.00,27.29,66.91,42.14,2.57,66.19,2.86,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2016-08-08,0.57,11.43,0.71,0.00,0.00,0.00,0.14,14.29,0.43,0.00,0.00,0.29,6.57,34.26,9.14,2.14,10.71,2.86,0.00,0.00,0.00,0.00,0.00,0.00,7.14,52.46,8.14,1.00,10.00,1.43,1.57,47.62,2.14,0.0,0.0,0.00,8.57,60.29,11.43,0.14,14.29,0.14,8.29,62.89,13.14,11.71,85.04,15.71,8.00,26.43,9.29,3.14,12.09,3.71,32.00,76.94,46.29,1.86,54.29,2.14,0.00,0.00,0.00,0.0,0.0,0.00,0.29,9.52,0.43,0.00,0.00,0.00
2016-08-09,0.71,16.19,1.14,0.00,0.00,0.14,0.14,14.29,0.71,0.00,0.00,0.57,11.57,43.52,16.86,4.29,25.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,9.71,63.17,11.57,2.57,19.24,3.86,3.00,59.52,3.86,0.0,0.0,0.14,15.71,70.65,21.29,0.14,14.29,0.43,11.57,66.81,16.43,11.71,79.75,17.71,17.29,37.89,20.86,7.14,24.21,8.43,33.86,77.86,48.43,1.29,42.86,1.71,0.00,0.00,0.00,0.0,0.0,0.00,0.43,16.67,0.71,0.00,0.00,0.00
2016-08-10,1.00,25.71,1.57,0.00,0.00,0.14,0.43,23.81,1.14,0.00,0.00,0.57,16.57,53.33,24.14,5.86,37.09,6.86,0.00,0.00,0.00,0.00,0.00,0.00,12.43,74.03,15.14,4.71,31.85,6.29,4.00,73.81,4.86,0.0,0.0,0.14,22.14,81.36,29.86,0.14,14.29,0.43,22.71,71.63,27.57,11.57,80.59,17.43,25.00,49.07,30.71,10.86,37.97,12.29,35.71,78.66,50.29,0.71,28.57,1.14,0.00,0.00,0.00,0.0,0.0,0.00,0.86,30.95,1.14,0.00,0.00,0.00
2016-08-11,1.29,31.43,2.29,0.00,0.00,0.14,0.71,29.52,1.86,0.00,0.00,0.57,22.43,62.93,32.86,9.29,51.37,10.29,0.00,0.00,0.00,0.00,0.00,0.00,15.14,84.47,18.86,8.00,46.13,9.57,5.00,83.81,6.29,0.0,0.0,0.14,25.86,76.26,35.71,0.14,14.29,0.43,30.71,79.04,35.86,13.43,82.02,19.14,30.86,59.17,39.00,15.14,52.25,16.57,38.71,77.46,56.00,0.29,14.29,0.71,0.00,0.00,0.00,0.0,0.0,0.00,1.29,38.10,2.00,0.00,0.00,0.00
2016-08-12,1.29,31.43,2.43,0.00,0.00,0.14,0.86,36.67,2.14,0.00,0.00,0.57,27.43,59.03,40.00,11.57,62.80,13.14,0.00,0.00,0.00,0.00,0.00,0.00,16.14,81.98,20.43,10.86,59.74,12.57,5.00,76.67,6.43,0.0,0.0,0.14,29.71,74.09,42.00,0.14,14.29,0.43,34.00,82.35,39.71,15.57,82.02,21.71,37.43,70.50,47.29,19.43,64.86,21.43,41.57,72.84,63.57,0.29,14.29,0.71,0.00,0.00,0.00,0.0,0.0,0.00,1.43,45.24,2.29,0.00,0.00,0.00
2016-08-13,1.43,38.57,2.71,0.00,0.00,0.14,1.00,43.81,2.14,0.00,0.00,0.57,27.86,59.11,40.57,11.86,72.33,13.57,0.00,0.00,0.00,0.00,0.00,0.00,16.71,80.08,21.57,11.71,74.03,13.43,5.43,79.64,6.71,0.0,0.0,0.14,30.71,76.43,42.43,0.14,14.29,0.43,35.29,81.53,41.71,14.00,78.28,20.14,39.29,80.28,50.00,20.43,79.14,22.43,43.29,68.43,67.57,0.29,14.29,0.71,0.00,0.00,0.00,0.0,0.0,0.00,1.43,45.24,2.43,0.00,0.00,0.00
2016-08-14,2.00,48.10,3.57,0.00,0.00,0.29,1.29,50.95,2.71,0.00,0.00,0.71,30.43,69.40,44.14,12.14,81.85,14.00,0.00,0.00,0.00,0.00,0.00,0.00,18.71,74.94,24.86,12.14,84.74,14.00,5.86,74.29,7.57,0.0,0.0,0.29,34.14,72.50,47.43,0.00,0.00,0.43,38.00,82.81,44.57,14.29,71.13,21.14,43.57,77.35,55.57,21.57,90.57,23.86,45.29,63

In [22]:
state_party_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2016-08-01 to 2016-11-08
Freq: D
Data columns (total 72 columns):
AZ_D_con                94 non-null float64
AZ_D_neg_ratio          94 non-null float64
AZ_D_tot                94 non-null float64
AZ_R_con                94 non-null float64
AZ_R_neg_ratio          94 non-null float64
AZ_R_tot                94 non-null float64
CO_D_con                94 non-null float64
CO_D_neg_ratio          94 non-null float64
CO_D_tot                94 non-null float64
CO_R_con                94 non-null float64
CO_R_neg_ratio          94 non-null float64
CO_R_tot                94 non-null float64
FL_D_con                94 non-null float64
FL_D_neg_ratio          94 non-null float64
FL_D_tot                94 non-null float64
FL_R_con                94 non-null float64
FL_R_neg_ratio          94 non-null float64
FL_R_tot                94 non-null float64
IA_D_con                94 non-null float64
IA_D_neg_ratio          94 non-n

## Pickling DataFrame to Use in a Separate Notebook for Visualizations

In [24]:
# state_party_df.to_pickle('data/state_party_neg_ad_mix_7DMA.pickle')